# Chapter 7: Wire Media

**PyMetaLearn - Metamaterial Physics Workbook**

---

## Learning Objectives

By the end of this chapter, you will:
- Understand how arrays of thin wires create ε < 0
- Derive the effective plasma frequency of wire media
- Calculate the effective permittivity of wire arrays
- Design wire media for specific plasma frequencies
- Understand the combination with SRRs for DNG materials

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, Markdown, Math

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# Physical constants
c = 3e8            # Speed of light (m/s)
mu_0 = 4 * np.pi * 1e-7  # Permeability of free space (H/m)
epsilon_0 = 8.854e-12    # Permittivity of free space (F/m)
e = 1.602e-19            # Elementary charge (C)
m_e = 9.109e-31          # Electron mass (kg)

print("Libraries loaded!")

---

## 7.1 The Idea: Artificial Plasma

Recall the **Drude model** for metals:

$$\varepsilon_r = 1 - \frac{\omega_p^2}{\omega^2 + i\gamma\omega}$$

Where the plasma frequency is:

$$\omega_p = \sqrt{\frac{n_e e^2}{\varepsilon_0 m_e}}$$

### The Problem

For real metals, $\omega_p$ is in the **UV** (hundreds of THz).

We want ε < 0 at **microwave or THz** frequencies!

### Pendry's Solution (1996)

Use a **dilute array of thin wires**. The effective electron density is much lower than bulk metal → lower effective plasma frequency.

In [ ]:
# Visualize wire medium geometry
fig = plt.figure(figsize=(14, 6))

# 2D cross-section
ax1 = fig.add_subplot(121)

# Draw wire cross-sections in a square lattice
a = 1.0  # Lattice constant (normalized)
r = 0.05  # Wire radius

for i in range(-2, 3):
    for j in range(-2, 3):
        circle = plt.Circle((i * a, j * a), r, color='blue', fill=True)
        ax1.add_patch(circle)

# Labels
ax1.annotate('', xy=(0, 0), xytext=(a, 0),
            arrowprops=dict(arrowstyle='<->', color='red', lw=2))
ax1.text(a/2, -0.3, 'a (lattice)', fontsize=11, ha='center', color='red')

ax1.annotate('', xy=(0, 0), xytext=(r, 0),
            arrowprops=dict(arrowstyle='<->', color='green', lw=1.5))
ax1.text(r/2, 0.15, 'r', fontsize=10, ha='center', color='green')

ax1.set_xlim(-2.5, 2.5)
ax1.set_ylim(-2.5, 2.5)
ax1.set_aspect('equal')
ax1.set_title('Wire Medium: Cross-Section\n(E ∥ to wires)', fontsize=12)
ax1.set_xlabel('x', fontsize=11)
ax1.set_ylabel('y', fontsize=11)
ax1.grid(True, alpha=0.3)

# 3D view
ax2 = fig.add_subplot(122, projection='3d')

# Draw wires as cylinders
z = np.linspace(-1, 1, 30)
theta = np.linspace(0, 2*np.pi, 20)
Z, THETA = np.meshgrid(z, theta)

for i in range(-1, 2):
    for j in range(-1, 2):
        X = r * np.cos(THETA) + i * a
        Y = r * np.sin(THETA) + j * a
        ax2.plot_surface(X, Y, Z, color='blue', alpha=0.7)

# Field direction
ax2.quiver(0, 1.5, 0, 0, 0, 0.8, color='red', arrow_length_ratio=0.2, linewidth=2)
ax2.text(0, 1.5, 1, 'E', fontsize=14, color='red')

ax2.set_xlim(-1.5, 1.5)
ax2.set_ylim(-1.5, 1.5)
ax2.set_zlim(-1, 1)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('z')
ax2.set_title('Wire Medium: 3D View', fontsize=12)

plt.tight_layout()
plt.show()

print("Key parameters:")
print("• a = lattice constant (wire spacing)")
print("• r = wire radius")
print("• Electric field E is parallel to the wires")

---

## 7.2 Effective Plasma Frequency

Pendry showed that a wire medium behaves like a Drude metal with an **effective plasma frequency**:

$$\boxed{\omega_p^{eff} = \sqrt{\frac{2\pi c^2}{a^2 \ln(a/r)}}}$$

Where:
- $a$ = lattice constant (wire spacing)
- $r$ = wire radius
- $c$ = speed of light

### Physical Understanding

1. **Dilution effect**: Only a small fraction of space is metal → lower effective electron density
2. **Kinetic inductance**: Electrons in thin wires have enhanced effective mass

The effective electron mass is:
$$m_{eff} = \frac{\mu_0 \pi r^2 n_e e^2}{2} \ln\left(\frac{a}{r}\right)$$

In [ ]:
def wire_medium_plasma_freq(a, r):
    """
    Calculate effective plasma frequency of wire medium.
    
    a: lattice constant (m)
    r: wire radius (m)
    
    Returns: omega_p in rad/s, f_p in Hz
    """
    omega_p = np.sqrt(2 * np.pi * c**2 / (a**2 * np.log(a/r)))
    f_p = omega_p / (2 * np.pi)
    return omega_p, f_p

# Design examples
print("Wire Medium Design Examples")
print("=" * 70)

designs = [
    {'name': 'Microwave (10 GHz)', 'a': 5e-3, 'r': 20e-6},
    {'name': 'mm-Wave (100 GHz)', 'a': 500e-6, 'r': 10e-6},
    {'name': 'THz (1 THz)', 'a': 50e-6, 'r': 1e-6},
    {'name': 'IR (10 THz)', 'a': 5e-6, 'r': 100e-9},
]

print(f"{'Design':<25} {'a (spacing)':<15} {'r (radius)':<15} {'f_p':<15}")
print("-" * 70)

for d in designs:
    omega_p, f_p = wire_medium_plasma_freq(d['a'], d['r'])
    
    # Format units
    if d['a'] >= 1e-3:
        a_str = f"{d['a']*1e3:.1f} mm"
    else:
        a_str = f"{d['a']*1e6:.0f} μm"
    
    if d['r'] >= 1e-6:
        r_str = f"{d['r']*1e6:.0f} μm"
    else:
        r_str = f"{d['r']*1e9:.0f} nm"
    
    if f_p >= 1e12:
        f_str = f"{f_p/1e12:.1f} THz"
    else:
        f_str = f"{f_p/1e9:.1f} GHz"
    
    print(f"{d['name']:<25} {a_str:<15} {r_str:<15} {f_str:<15}")

print("\n💡 Larger spacing → lower plasma frequency")

---

## 7.3 Effective Permittivity

The effective permittivity follows the Drude form:

$$\boxed{\varepsilon_{eff}(\omega) = 1 - \frac{(\omega_p^{eff})^2}{\omega^2 + i\gamma\omega}}$$

Where $\gamma$ accounts for resistive losses in the wires.

### Key Points

1. For $\omega < \omega_p^{eff}$: $\varepsilon_{eff} < 0$ (like a metal!)
2. For $\omega > \omega_p^{eff}$: $\varepsilon_{eff} > 0$ (transparent)
3. The transition frequency is controlled by geometry, not material

In [ ]:
def wire_medium_permittivity(omega, omega_p, gamma):
    """Calculate effective permittivity of wire medium."""
    eps_eff = 1 - omega_p**2 / (omega**2 + 1j * gamma * omega)
    return eps_eff

# Example: Design for 10 GHz plasma frequency
a = 5e-3  # 5 mm spacing
r = 20e-6  # 20 μm radius
omega_p, f_p = wire_medium_plasma_freq(a, r)
gamma = omega_p * 0.02  # 2% damping

# Frequency range
freq = np.linspace(0.1, 3, 500) * f_p  # 0.1 to 3 times plasma frequency
omega = 2 * np.pi * freq

eps = wire_medium_permittivity(omega, omega_p, gamma)

# Plotting
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Real part
ax1 = axes[0]
ax1.plot(freq / 1e9, eps.real, 'b-', linewidth=2)
ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax1.axvline(x=f_p / 1e9, color='red', linestyle=':', linewidth=2, label=r'$f_p$')
ax1.fill_between(freq / 1e9, eps.real, 0, where=(eps.real < 0), 
                 alpha=0.3, color='red', label='ε < 0')

ax1.set_xlabel('Frequency (GHz)', fontsize=14)
ax1.set_ylabel(r"$\varepsilon'$ (Real Part)", fontsize=14)
ax1.set_title(f'Wire Medium Permittivity\na = {a*1e3} mm, r = {r*1e6} μm', fontsize=14)
ax1.set_ylim(-20, 5)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Imaginary part
ax2 = axes[1]
ax2.plot(freq / 1e9, eps.imag, 'r-', linewidth=2)
ax2.axvline(x=f_p / 1e9, color='red', linestyle=':', linewidth=2, label=r'$f_p$')
ax2.set_xlabel('Frequency (GHz)', fontsize=14)
ax2.set_ylabel(r"$\varepsilon''$ (Imaginary Part)", fontsize=14)
ax2.set_title('Losses in Wire Medium', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Plasma frequency: f_p = {f_p/1e9:.2f} GHz")
print(f"Below {f_p/1e9:.1f} GHz: ε < 0 (metallic behavior)")
print(f"Above {f_p/1e9:.1f} GHz: ε > 0 (dielectric behavior)")

---

## 7.4 Design Parameters

How do the wire parameters affect the plasma frequency?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Effect of lattice constant
ax1 = axes[0]
r_fixed = 20e-6  # Fixed wire radius
a_values = np.linspace(1e-3, 20e-3, 100)

f_p_values = [wire_medium_plasma_freq(a, r_fixed)[1] for a in a_values]
ax1.plot(a_values * 1e3, np.array(f_p_values) / 1e9, 'b-', linewidth=2)
ax1.set_xlabel('Lattice Constant a (mm)', fontsize=14)
ax1.set_ylabel('Plasma Frequency (GHz)', fontsize=14)
ax1.set_title(f'Effect of Wire Spacing (r = {r_fixed*1e6} μm)', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Effect of wire radius
ax2 = axes[1]
a_fixed = 5e-3  # Fixed lattice constant
r_values = np.linspace(1e-6, 200e-6, 100)

f_p_values = [wire_medium_plasma_freq(a_fixed, r)[1] for r in r_values if r < a_fixed]
r_valid = [r for r in r_values if r < a_fixed]
ax2.plot(np.array(r_valid) * 1e6, np.array(f_p_values) / 1e9, 'r-', linewidth=2)
ax2.set_xlabel('Wire Radius r (μm)', fontsize=14)
ax2.set_ylabel('Plasma Frequency (GHz)', fontsize=14)
ax2.set_title(f'Effect of Wire Radius (a = {a_fixed*1e3} mm)', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Design rules:")
print("• Larger spacing a → Lower plasma frequency (1/a dependence)")
print("• Thinner wires r → Lower plasma frequency (logarithmic dependence)")
print("• Constraint: r << a (dilute limit)")

---

## 7.5 Comparison with Real Metals

Let's compare the wire medium plasma frequency with real metals.

In [ ]:
# Real metal plasma frequencies
metals = {
    'Silver': 5.86e28,
    'Gold': 5.90e28,
    'Copper': 8.47e28,
    'Aluminum': 18.1e28,
}

print("Real Metals vs Wire Media")
print("=" * 70)
print(f"{'Material':<20} {'Electron Density':<20} {'Plasma Freq':<15} {'Region':<15}")
print("-" * 70)

for metal, n_e in metals.items():
    omega_p = np.sqrt(n_e * e**2 / (epsilon_0 * m_e))
    f_p = omega_p / (2 * np.pi)
    print(f"{metal:<20} {n_e:.2e} m⁻³    {f_p/1e15:.2f} PHz    UV")

print("-" * 70)

# Wire media
wire_designs = [
    ('Wire (a=5mm)', 5e-3, 20e-6),
    ('Wire (a=1mm)', 1e-3, 10e-6),
    ('Wire (a=100μm)', 100e-6, 5e-6),
]

for name, a, r in wire_designs:
    omega_p, f_p = wire_medium_plasma_freq(a, r)
    if f_p > 1e12:
        f_str = f"{f_p/1e12:.2f} THz"
        region = "IR/THz"
    elif f_p > 1e9:
        f_str = f"{f_p/1e9:.2f} GHz"
        region = "Microwave"
    else:
        f_str = f"{f_p/1e6:.2f} MHz"
        region = "RF"
    print(f"{name:<20} {'artificial':<20} {f_str:<15} {region:<15}")

print("\n💡 Wire media can achieve metallic (ε < 0) response at much lower frequencies!")

---

## 7.6 Wave Propagation in Wire Media

Let's visualize how electromagnetic waves interact with wire media.

In [ ]:
# Wave propagation: below and above plasma frequency
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

x = np.linspace(0, 5, 500)
omega_p = 1.0  # Normalized plasma frequency

# Below plasma frequency (ε < 0): evanescent
omega_low = 0.5 * omega_p
eps_low = wire_medium_permittivity(omega_low, omega_p, 0.01)
k_low = omega_low * np.sqrt(eps_low.real) / c if eps_low.real > 0 else 0
kappa = omega_low * np.sqrt(-eps_low.real) / c if eps_low.real < 0 else 0

ax1 = axes[0, 0]
ax1.axvspan(0, 1, alpha=0.3, color='gray', label='Wire medium')
wave_evanescent = np.exp(-2 * (x - 1)) * np.where(x >= 1, 1, 0) + np.cos(4 * x) * np.where(x < 1, 1, 0)
ax1.plot(x, wave_evanescent, 'b-', linewidth=2)
ax1.set_ylabel('Field Amplitude', fontsize=12)
ax1.set_title(f'Below Plasma Freq (ω = 0.5ωₚ)\nε = {eps_low.real:.2f} → Evanescent', fontsize=12)
ax1.set_ylim(-1.5, 1.5)
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Above plasma frequency (ε > 0): propagating
omega_high = 2.0 * omega_p
eps_high = wire_medium_permittivity(omega_high, omega_p, 0.01)

ax2 = axes[0, 1]
ax2.axvspan(0, 1, alpha=0.3, color='lightblue', label='Wire medium')
wave_prop = np.cos(4 * x) * np.where(x < 1, 1, 0) + np.cos(4 * np.sqrt(eps_high.real) * (x - 1) + 4) * np.where(x >= 1, 1, 0)
ax2.plot(x, np.real(wave_prop), 'g-', linewidth=2)
ax2.set_ylabel('Field Amplitude', fontsize=12)
ax2.set_title(f'Above Plasma Freq (ω = 2ωₚ)\nε = {eps_high.real:.2f} → Propagating', fontsize=12)
ax2.set_ylim(-1.5, 1.5)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Dispersion relation
ax3 = axes[1, 0]
omega_range = np.linspace(0.01, 3, 200)
eps = wire_medium_permittivity(omega_range, 1.0, 0.02)
n = np.sqrt(eps)
k_real = omega_range * np.real(n)
k_imag = omega_range * np.imag(n)

ax3.plot(k_real, omega_range, 'b-', linewidth=2, label="k' (propagating)")
ax3.plot(k_imag, omega_range, 'r--', linewidth=2, label="k'' (evanescent)")
ax3.axhline(y=1, color='green', linestyle=':', linewidth=2, label=r'$\omega_p$')
ax3.plot([0, 3], [0, 3], 'gray', linestyle='--', alpha=0.5, label='Light line')
ax3.set_xlabel('Wavevector k (normalized)', fontsize=12)
ax3.set_ylabel('Frequency ω/ωₚ', fontsize=12)
ax3.set_title('Dispersion Relation', fontsize=12)
ax3.set_xlim(0, 3)
ax3.set_ylim(0, 3)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Transmission vs frequency
ax4 = axes[1, 1]
freq_range = np.linspace(0.1, 3, 200)
eps = wire_medium_permittivity(freq_range, 1.0, 0.02)
n = np.sqrt(eps)

# Fresnel coefficients (normal incidence from air)
r = (1 - n) / (1 + n)
t = 2 / (1 + n)
R = np.abs(r)**2 * 100
T = np.abs(t)**2 * np.real(n) * 100
T[np.real(n) <= 0] = 0  # No transmission when evanescent

ax4.plot(freq_range, R, 'r-', linewidth=2, label='Reflectance')
ax4.plot(freq_range, T, 'b-', linewidth=2, label='Transmittance')
ax4.axvline(x=1, color='green', linestyle=':', linewidth=2, label=r'$\omega_p$')
ax4.fill_between(freq_range, 0, 100, where=(freq_range < 1), alpha=0.1, color='red')
ax4.set_xlabel('Frequency ω/ωₚ', fontsize=12)
ax4.set_ylabel('Percentage', fontsize=12)
ax4.set_title('Reflectance & Transmittance', fontsize=12)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Below ωₚ: High reflectance, near-zero transmission (like a metal)")
print("Above ωₚ: Transmission increases, reflectance decreases (like a dielectric)")

---

## 7.7 Combining with SRRs for DNG

The wire medium provides ε < 0. Combined with SRRs (μ < 0), we get a **Double Negative (DNG)** metamaterial.

### Design Requirements

For DNG behavior, we need the frequency bands to overlap:

- Wire medium: ε < 0 for ω < ω_p^wire
- SRR: μ < 0 for ω_0^SRR < ω < ω_mp^SRR

**Overlap condition**: $\omega_0^{SRR} < \omega_p^{wire}$

In [ ]:
# Combined SRR + Wire medium
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

omega = np.linspace(0.01, 2.5, 500)

# Wire medium parameters
omega_p_wire = 1.8
gamma_wire = 0.03

# SRR parameters
omega_0_srr = 1.0
F = 0.3
gamma_srr = 0.05

# Calculate effective parameters
eps_eff = wire_medium_permittivity(omega, omega_p_wire, gamma_wire)

# SRR effective permeability (from Chapter 6)
mu_eff = 1 - F * omega**2 / (omega**2 - omega_0_srr**2 + 1j * gamma_srr * omega)

# Find DNG region
dng_mask = (eps_eff.real < 0) & (mu_eff.real < 0)

# Plot effective parameters
ax1 = axes[0]
ax1.plot(omega, eps_eff.real, 'b-', linewidth=2, label=r"$\varepsilon'$ (wire)")
ax1.plot(omega, mu_eff.real, 'r-', linewidth=2, label=r"$\mu'$ (SRR)")
ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax1.fill_between(omega, -10, 5, where=dng_mask, alpha=0.2, color='green', label='DNG region')

ax1.axvline(x=omega_0_srr, color='red', linestyle=':', alpha=0.5, label=r'$\omega_0^{SRR}$')
ax1.axvline(x=omega_p_wire, color='blue', linestyle=':', alpha=0.5, label=r'$\omega_p^{wire}$')

ax1.set_xlabel(r'$\omega$ (normalized)', fontsize=14)
ax1.set_ylabel('Effective Parameters', fontsize=14)
ax1.set_title('SRR + Wire Medium = DNG Metamaterial', fontsize=14)
ax1.set_xlim(0, 2.5)
ax1.set_ylim(-10, 5)
ax1.legend(loc='lower right', fontsize=9)
ax1.grid(True, alpha=0.3)

# Calculate and plot refractive index
ax2 = axes[1]
n_squared = eps_eff * mu_eff
n_eff = np.sqrt(n_squared)
n_real = np.real(n_eff)

# In DNG region, n should be negative
n_real_corrected = n_real.copy()
n_real_corrected[dng_mask] = -np.abs(n_real_corrected[dng_mask])

ax2.plot(omega, n_real_corrected, 'purple', linewidth=2, label="n (real part)")
ax2.plot(omega, np.imag(n_eff), 'orange', linewidth=2, linestyle='--', label="n (imaginary)")
ax2.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax2.fill_between(omega, -4, 0, where=(n_real_corrected < 0), alpha=0.2, color='purple', label='n < 0')

ax2.set_xlabel(r'$\omega$ (normalized)', fontsize=14)
ax2.set_ylabel('Effective Refractive Index', fontsize=14)
ax2.set_title('Negative Index Band', fontsize=14)
ax2.set_xlim(0, 2.5)
ax2.set_ylim(-4, 4)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find DNG bandwidth
dng_freqs = omega[dng_mask]
if len(dng_freqs) > 0:
    print(f"DNG region: {dng_freqs.min():.2f} < ω < {dng_freqs.max():.2f}")
    bw = (dng_freqs.max() - dng_freqs.min()) / omega_0_srr * 100
    print(f"Bandwidth: {bw:.1f}% of ω₀")

---

## 7.8 Summary & Key Takeaways

### What We Learned

1. **Wire media** create artificial plasma with tunable plasma frequency
2. **Effective plasma frequency**: $\omega_p^{eff} = \sqrt{2\pi c^2 / (a^2 \ln(a/r))}$
3. For $\omega < \omega_p^{eff}$: **ε < 0** (metallic behavior)
4. Larger spacing and thinner wires → lower plasma frequency
5. **Combined with SRRs** → DNG metamaterial with n < 0

### The Complete Picture

| Component | Provides | Frequency Dependence |
|-----------|----------|---------------------|
| Wire medium | ε < 0 | Below plasma frequency |
| SRR array | μ < 0 | Narrow band above resonance |
| Combined | n < 0 | Where both bands overlap |

### Preview of Next Chapter

In **Chapter 8: Optical Metamaterials**, we will:
- Explore fishnet and other optical-frequency designs
- Understand scaling challenges at optical frequencies
- Learn about plasmonic metamaterials

---

## 📝 Exercises

1. **Design** a wire medium with plasma frequency at 5 GHz
2. **Calculate** the effective mass enhancement for electrons in wires
3. **Show** that reducing wire radius lowers the plasma frequency
4. **Design** a combined SRR + wire medium for n = -1 at 10 GHz
5. **Research**: What limits the scalability of wire media to optical frequencies?

---

**← [06 - Split-Ring Resonators](06_srr.ipynb) | [08 - Optical Metamaterials](08_optical_metamaterials.ipynb) →**